## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [35]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()
print(y_df['Attrition'].value_counts())

Attrition
No     1233
Yes     237
Name: count, dtype: int64


In [4]:
# Create a list of at least 10 column names to use as X data
column_names = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
            'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany',
            'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[column_names]


# Show the data types for X_df
X_df.dtypes

,0
Education,int64
Age,int64
DistanceFromHome,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [9]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)


In [17]:
# Convert your X data to numeric data types however you see fit
# conert the OverTime column to numeric data type
# label encode the overtime column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train['OverTime'] = le.fit_transform(X_train['OverTime'])
X_test['OverTime'] = le.transform(X_test['OverTime'])

X_train['OverTime'].value_counts()


,count
OverTime,
0,798
1,304


In [18]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)


In [19]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [20]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
enc = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = enc.fit_transform(y_train[['Department']])
department_test_encoded = enc.transform(y_test[['Department']])




/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
# Create a OneHotEncoder for the Attrition column
enc_attrition = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = enc_attrition.fit_transform(y_train[['Attrition']])
attrition_test_encoded = enc_attrition.transform(y_test[['Attrition']])



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [23]:
# Find the number of columns in the X training data
input_nodes = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_nodes,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [27]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(64, activation='relu')(shared_layer2)


# Create the output layer
department_output = layers.Dense(department_train_encoded.shape[1],
                             activation='softmax',
                             name='department_output')(department_dense)


In [28]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(64, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(attrition_train_encoded.shape[1],
                             activation='sigmoid',
                             name='attrition_output')(attrition_dense)


In [29]:
# Create the model

model = Model(inputs=input_layer, outputs=[
    department_output,
    attrition_output,
])

# Compile the Model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy',
                       'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 10)]                 0         []                            
 )                                                                                                
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_features[0][0]']      
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_3 (Dense)             (None, 64)                   2112      ['dense_1[0][0]']         

In [30]:
# Train the model
model.fit(X_train_scaled,
          {'department_output': department_train_encoded,
           'attrition_output': attrition_train_encoded},
          epochs=100,
          batch_size=32,
          validation_data=(X_test_scaled,
                           {'department_output': department_test_encoded,
                            'attrition_output': attrition_test_encoded}))


Epoch 1/100
35/35 [==============================] - 3s 25ms/step - loss: 1.5837 - department_output_loss: 0.9322 - attrition_output_loss: 0.6515 - department_output_accuracy: 0.5672 - attrition_output_accuracy: 0.6534 - val_loss: 1.3363 - val_department_output_loss: 0.8030 - val_attrition_output_loss: 0.5333 - val_department_output_accuracy: 0.6413 - val_attrition_output_accuracy: 0.8152
Epoch 2/100
35/35 [==============================] - 0s 8ms/step - loss: 1.2392 - department_output_loss: 0.7872 - attrition_output_loss: 0.4519 - department_output_accuracy: 0.6579 - attrition_output_accuracy: 0.8466 - val_loss: 1.2558 - val_department_output_loss: 0.7810 - val_attrition_output_loss: 0.4749 - val_department_output_accuracy: 0.6413 - val_attrition_output_accuracy: 0.8152
Epoch 3/100
35/35 [==============================] - 0s 9ms/step - loss: 1.1804 - department_output_loss: 0.7737 - attrition_output_loss: 0.4066 - department_output_accuracy: 0.6579 - attrition_output_accuracy: 0.8466

In [31]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled,
                         {'department_output': department_test_encoded,
                          'attrition_output': attrition_test_encoded})

12/12 [==============================] - 0s 3ms/step - loss: 3.5970 - department_output_loss: 2.4009 - attrition_output_loss: 1.1961 - department_output_accuracy: 0.5598 - attrition_output_accuracy: 0.7853


In [33]:
# Print the accuracy for both department and attrition
print(f'Department Accuracy: {results[3]}')
print(f'Attrition Accuracy: {results[4]}')

Department Accuracy: 0.5597826242446899
Attrition Accuracy: 0.7853260636329651


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?


2. What activation functions did you choose for your output layers, and why?


3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Considering we have imblance in our data (e.g. in the Attrition values) and the Department Accuracy is only 55%, I'd say we would want to consider other metrics, the likes of which would be found in a classification report. This would provide more details about the overall balance of the model, its ability to predict the majority vs. minority classes, etc.

2. I chose sigmoid for the binary outcomes in Attrition, whereas softmax is able to handle the multi-classification required for the Department values.

3. Considering my inexperience with this, it's fair to say the model can be improved by making optimizations from preprocessing all the way through evaluating the results. There could be more analysis required to refine the number of epochs, the number of dense layers and their activation functions, which features are used in training, encoding strategy, etc. There is so much complexity to these models vs. our earlier models that the number of refinements that could be made is significantly greater.